In [16]:
import numpy as np
import tensorflow as tf
import pretty_midi
import pathlib
import glob

class notenClass:
    def __init__(self, toonHoogte, beginTijd, eindTijd, nootInterval, nootTijd):
        self.toonHoogte = toonHoogte
        self.beginTijd = beginTijd
        self.eindTijd = eindTijd
        self.nootInterval = nootInterval
        self.nootTijd = nootTijd

# Zet MIDI bestand om in een array van noten met hun bijhorende eigenschappen (bijvoorbeeld: toonhoogte).
def midiNaarNoten(bestand):
    noten = notenClass([], [], [], [], [])
    piano = pretty_midi.PrettyMIDI(bestand).instruments[0]

    # Rangschik alle gespeelde noten door de piano op chronologische volgorde.
    notenGerangschikt = sorted(piano.notes, key = (lambda noot : noot.start))

    # Orden alle noten met haar eigenschappen in de noten class.
    beginVorigeNoot = notenGerangschikt[0].start
    for noot in notenGerangschikt:
        noten.toonHoogte.append(noot.pitch)
        noten.beginTijd.append(noot.start)
        noten.eindTijd.append(noot.end)
        noten.nootInterval.append(noot.start - beginVorigeNoot)
        noten.nootTijd.append(noot.end - noot.start)
        beginVorigeNoot = noot.start
    
    # Creeër een array 
    notenArray = np.stack((noten.toonHoogte, noten.beginTijd, noten.eindTijd, noten.nootInterval, noten.nootTijd), axis = 1)
    
    return notenArray

# Normalizeer toonhoogte naar de toonhoogten die ondersteund worden door Pretty-midi.
def normalizeerToonHoogte(notenArray):
    notenArray = notenArray/[maxToonHoogte, 1.0, 1.0]
    return notenArray

# Zet een array van noten met haar bijhorende eigenschappen om in een MIDI bestand.
def notenNaarMidi(noten, exportBestand, instrumentNaam = 'Acoustic Grand Piano', geluidsterkte = 100):
    bestand = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program(instrumentNaam))

    # Zet de array van noten om in Pretty_MIDI noten en voeg deze toe aan het gegeven instrument.
    startVorigeNoot = 0
    for noot in noten:
        nootInterval = noot[3]
        nootTijd = noot[4]
        beginTijd = float(startVorigeNoot + nootInterval)
        eindTijd = float(beginTijd + nootTijd)
        noot = pretty_midi.Note(velocity = geluidsterkte, pitch = int(noot[0]), start = beginTijd, end = eindTijd)
        instrument.notes.append(noot)
        startVorigeNoot = beginTijd

    bestand.instruments.append(instrument)

    # Exporteer de Pretty_MIDI object naar een .midi/.mid bestand in de directory van dit programma.
    bestand.write(exportBestand)
    return bestand

# Deelt dataset op in batches met labels en genormalizeerde inputs.
def verwerkDataset(dataset, batchInputGrootte):
    batchInputGrootte += 1

    # Maak een nieuwe dataset die windows bevat van de dataset uit de input.
    windows = dataset.window(batchInputGrootte, shift = 1, drop_remainder = True)

    # Zet de windows om in batches en 'flatten' deze batches tot een nieuwe dataset.
    flattenWindows = lambda window: window.batch(batchInputGrootte, drop_remainder = True)
    flattenedDataset = windows.flat_map(flattenWindows)

    # Deel batch op in inputs en een bijbehorend label. En normalizeer de inputs. 
    def deelNormalizeerBatches(notenArray):
        inputs = notenArray[:-1]
        labels = notenArray[-1]
        labelsDictionary = {}
        labelsDictionary.update({'toonHoogte': labels[0]})
        labelsDictionary.update({'nootInterval': labels[1]})
        labelsDictionary.update({'nootTijd': labels[2]})

        return normalizeerToonHoogte(inputs), labelsDictionary

    # Pas bovenstaande functie toe tot elke batch in de dataset.
    # 2e argument in Dataset.map(), laat deze bovenstaande functie parallel toegepast worden door CPU.
    notenDataset = flattenedDataset.map(deelNormalizeerBatches, num_parallel_calls = tf.data.AUTOTUNE)

    return notenDataset

# Dit is de cost-function. De meanSquaredError is een veelgebruikte cost-function binnen Deep Learning.
# Bij een grote verschil tussen de waarde en de verwachte waarde, wordt de 'fout' groot.
# Een klein verschil resulteert tot een kleine 'fout'.
def costFunction(waarde, verwachteWaarde):
    meanSquaredError = (waarde - verwachteWaarde) ** 2

    # Maakt de waarde positief.
    positieveWaarde = tf.maximum(-verwachteWaarde, 0.0) * 10

    return tf.reduce_mean(meanSquaredError + positieveWaarde)

# Genereer nieuwe noot.
def maakNieuweNoot(noten, keras_model, temperatuur):
    inputNoten = tf.expand_dims(noten, 0)
    gemaakteNoten = netwerk.predict(inputNoten)
    preToonHoogte = gemaakteNoten['toonHoogte']
    nootInterval = gemaakteNoten['nootInterval']
    nootTijd = gemaakteNoten['nootTijd']
    preToonHoogte /= temperatuur
    toonHoogte = tf.random.categorical(preToonHoogte, num_samples = 1)
    toonHoogte = tf.squeeze(toonHoogte, axis = -1)
    nootTijd = tf.squeeze(nootTijd, axis = -1)
    nootInterval = tf.squeeze(nootInterval, axis = -1)
    nootInterval = tf.maximum(0, nootInterval)
    nootTijd = tf.maximum(0, nootTijd)

    return int(toonHoogte), float(nootInterval), float(nootTijd)

# Exporteer het nieuwe muziekstuk naar een MIDI file in de directory van dit programma.
def exporteerBestand(bestandsNaam):
    bestand = bestandsNaam + '.midi'
    exportMidi = notenNaarMidi(nieuweNoten, bestand)

# Maximale toonhoogte die Pretty-midi ondersteunt.
maxToonHoogte = 128

In [17]:
# Directory van onze dataset.
directory = pathlib.Path('data/bach/bach')

# Bestanden binnen onze dataset.
bestanden = []
for i in glob.glob('data/bach/bach/*/*[.mid]*'):
    bestanden.append(i)

bestanden = glob.glob(str(directory/'**/*.mid*'))

aantalBestanden = 5
alleNoten = []

# Orden alle noten met haar eigenschappen van het gebruikte aantal muziekstukken in een array.
for bestand in bestanden[:aantalBestanden]:
    noten = midiNaarNoten(bestand)
    alleNoten.append(noten)

notenArray = np.concatenate(alleNoten)
nAlleNoten = len(notenArray)

# Verwijder de begin- en eindtijden voor alle noten.
trainingsNoten = np.delete(notenArray, [1, 2], 1)

# Zet de array van trainingsnoten om in een dataset voor Tensorflow.
notenDataset = tf.data.Dataset.from_tensor_slices(trainingsNoten)

# Hoeveelheid inputs in de batches.
batchInputGrootte = 25

verwerkteDataset = verwerkDataset(notenDataset, batchInputGrootte)

# Deel batches op in meerdere batches, en hussel ze daarna voor nauwkeuriger trainen.
batchGrootte = 64
bufferGrootte = nAlleNoten - batchInputGrootte
trainingDataset = verwerkteDataset.shuffle(bufferGrootte).batch(batchGrootte, drop_remainder = True).cache().prefetch(tf.data.experimental.AUTOTUNE)
input = (batchInputGrootte, 3)

# Hyperparameter: bepaald de willekeur in voorspellingen van het netwerk.
# Des te groter deze hyperparameter, des te meer willekeur zal zitten in de voorspellingen van het netwerk.
temperatuur = 3.4

# A.k.a learning rate, parameter die bepaald hoe grote stappen de gradient descent maakt.
leerVelocity = 0.002

# Aantal keer dat de hele trainingset heen en terug door het netwerk gaat.
# Oftewel het aantal keer dat het network over de gehele trainingset wordt getraind.
epochs = 100

# Hoeveelheid noten die het netwerk moet genereren.
nNieuweNoten = 50
inputs = tf.keras.Input(input)

# De RNN input-layers.
lstm = tf.keras.layers.LSTM(128)(inputs)

# De RNN output-layers
outputs = {
              'toonHoogte': tf.keras.layers.Dense(128, name = 'toonHoogte')(lstm),
              'nootInterval': tf.keras.layers.Dense(1, name = 'nootInterval')(lstm),
              'nootTijd': tf.keras.layers.Dense(1, name = 'nootTijd')(lstm)
          }

# Maak netwerk object uit deze layers.
netwerk = tf.keras.Model(inputs, outputs)

# Cost-functions voor de elementen van de output.
cost = {
            'toonHoogte': tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits = True),
            'nootInterval': costFunction,
            'nootTijd': costFunction,
        }

# Algoritme dat de weights en biases zo aanpast om een local/global minimum te vinden.
# Hier wordt de stochastic gradient descent gebruikt.
minimumAlgoritme = tf.keras.optimizers.SGD(learning_rate = leerVelocity)
netwerk.compile(loss = cost, optimizer = minimumAlgoritme)

# De loss_weights geven aan hoeveel belang de cost-function stelt aan deze variabelen van een noot.
netwerk.compile(loss = cost, loss_weights = {'toonHoogte': 0.05, 'nootInterval': 1.0, 'nootTijd': 1.0}, optimizer =  minimumAlgoritme)

# Sla checkpoint van het trainen op in de directory van dit programma.
# Stop het trainen, wanneer de cost van een bepaalde output-waarde niet meer verbeterd.
utilityFuncties =   [
                        tf.keras.callbacks.ModelCheckpoint(filepath = './training_checkpoints/ckpt_{epoch}', save_weights_only = True),
                        tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 5, verbose = 1, restore_best_weights = True)
                    ]

# Train het netwerk.
train = netwerk.fit(trainingDataset, epochs = epochs, callbacks = utilityFuncties)

# Input noten voor het creeëren van een muziekstuk.
testNoten = midiNaarNoten(bestanden[1])
gecorrigeerdeTestNoten = np.delete(testNoten, [1, 2], 1)
nieuweInput = normalizeerToonHoogte(gecorrigeerdeTestNoten[:batchInputGrootte])

gemaakteNoten = []
beginVorigeNoot = 0

# Verwerk de gemaakte/voorspelde noten in een array.
for i in range(nNieuweNoten):
  toonHoogte, nootInterval, nootTijd = maakNieuweNoot(nieuweInput, netwerk, temperatuur)
  beginTijd = beginVorigeNoot + nootInterval
  eindTijd = beginVorigeNoot + nootTijd
  gemaakteNoot = (toonHoogte, nootInterval, nootTijd)
  gemaakteNoten.append((*gemaakteNoot, beginTijd, eindTijd))
  nieuweInput = np.delete(nieuweInput, 0, axis=0)
  nieuweInput = np.append(nieuweInput, np.expand_dims(gemaakteNoot, 0), axis=0)
  beginVorigeNoot = beginTijd

nieuweNoten = np.array(gemaakteNoten)

exporteerBestand('product')

Epoch 1/100
55/55 [==============================] - 6s 52ms/step - loss: 0.4623 - nootInterval_loss: 0.0674 - nootTijd_loss: 0.1518 - toonHoogte_loss: 4.8628
Epoch 2/100
55/55 [==============================] - 5s 83ms/step - loss: 0.4126 - nootInterval_loss: 0.0511 - nootTijd_loss: 0.1183 - toonHoogte_loss: 4.8627
Epoch 3/100
30/55 [===============>..............] - ETA: 2s - loss: 0.4183 - nootInterval_loss: 0.0528 - nootTijd_loss: 0.1224 - toonHoogte_loss: 4.8619

KeyboardInterrupt: 